In [1]:
#source https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset/data

## **Data Preprocessing**

In [1]:
import pandas as pd
import numpy as np

In [2]:
true_df = pd.read_csv('True.csv', delimiter=',')
fake_df = pd.read_csv('Fake.csv', delimiter=',')

In [3]:
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [4]:
true_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


In [5]:
true_df.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [6]:
true_df.describe(include ='all')

,title,text,subject,date
count,21417,21417,21417,21417
unique,20826,21192,2,716
top,Factbox: Trump fills top jobs for his administ...,(Reuters) - Highlights for U.S. President Dona...,politicsNews,"December 20, 2017"
freq,14,8,11272,182


In [8]:
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [9]:
fake_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


In [10]:
fake_df.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [11]:
fake_df.describe(include ='all')

,title,text,subject,date
count,23481,23481,23481,23481
unique,17903,17455,6,1681
top,MEDIA IGNORES Time That Bill Clinton FIRED His...,,News,"May 10, 2017"
freq,6,626,9050,46


In [12]:
true_df["label"] = 1 
fake_df["label"] = 0 

In [13]:
df = pd.concat([true_df, fake_df], axis=0).reset_index(drop=True)
df.drop(columns=['subject', "date"], inplace=True)
df

,title,text,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,1
...,...,...,...
44893,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,0
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,0
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,0
44896,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,0


In [14]:
empty_titles = df[df["title"].str.strip() == ""]
empty_titles

,title,text,label


## **Multinomial Naive Bayes Training**

In [15]:
!pip install scikit-learn

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import re

In [39]:
# Custom preprocessor to separate numbers from words
def custom_preprocessor(text):
    # Add a space between a number and the word following it
    text = re.sub(r'(\d)([a-zA-Z])', r'\1 \2', text)
    return text

In [40]:
# Select text and labels
X_text = df["text"]  # Features (news article content)
y = df["label"]  # Labels (1 = Real, 0 = Fake)

# Convert text into numerical features using TF-IDF
vectorizer = TfidfVectorizer(stop_words="english", preprocessor = custom_preprocessor) #token_pattern=r'\b\w+\b')
X = vectorizer.fit_transform(X_text)

In [41]:
# 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)


In [42]:
# Train Naïve Bayes Model
model = MultinomialNB(alpha = 1)
model.fit(X_train, y_train)

MultinomialNB(alpha=1)

## **Testing/Results**

In [ ]:
# Get log probabilities learned by Naïve Bayes
log_probs = model.feature_log_prob_

# Convert log probabilities to actual probabilities
probs = np.exp(log_probs)

# Get vocabulary (word list)
feature_names = vectorizer.get_feature_names_out()
priors = np.exp(model.class_log_prior_)

# Print actual prior probabilities
print("Prior Probabilities (P(Y)):")
print(f"P(Real) = {priors[1]:.4f}")
print(f"P(Fake) = {priors[0]:.4f}")
# View probabilities of first 100 words(vectorized)
for i in range(100):
    print(f"Word: {feature_names[i]}")
    print(f"P(word | Fake): {probs[0, i]:.6f}")
    print(f"P(word | Real): {probs[1, i]:.6f}")
    print("------")

Prior Probabilities (P(Y)):
P(Real) = 0.4763
P(Fake) = 0.5237
Word: 00
P(word | Fake): 0.000099
P(word | Real): 0.000013
------
Word: 000
P(word | Fake): 0.000300
P(word | Real): 0.000394
------
Word: 0000
P(word | Fake): 0.000004
P(word | Real): 0.000005
------
Word: 00000017
P(word | Fake): 0.000003
P(word | Real): 0.000003
------
Word: 00004
P(word | Fake): 0.000003
P(word | Real): 0.000003
------
Word: 000048
P(word | Fake): 0.000003
P(word | Real): 0.000003
------
Word: 000063
P(word | Fake): 0.000003
P(word | Real): 0.000004
------
Word: 00007
P(word | Fake): 0.000003
P(word | Real): 0.000003
------
Word: 000270
P(word | Fake): 0.000003
P(word | Real): 0.000003
------
Word: 00042
P(word | Fake): 0.000003
P(word | Real): 0.000004
------
Word: 0005
P(word | Fake): 0.000003
P(word | Real): 0.000003
------
Word: 0009
P(word | Fake): 0.000003
P(word | Real): 0.000004
------
Word: 000938
P(word | Fake): 0.000003
P(word | Real): 0.000004
------
Word: 001
P(word | Fake): 0.000006
P(word 

In [56]:
# Predict labels for the test set
y_pred = model.predict(X_test)

# Compute accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")

# Print classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Print confusion matrix
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Model Accuracy: 0.9501

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.96      0.95      4670
           1       0.95      0.94      0.95      4310

    accuracy                           0.95      8980
   macro avg       0.95      0.95      0.95      8980
weighted avg       0.95      0.95      0.95      8980


Confusion Matrix:
 [[4469  201]
 [ 247 4063]]
